In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
import imgaug as ia
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
import warnings
warnings.filterwarnings("ignore")

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow import set_random_seed
set_random_seed(42)
np.random.seed(42)
ia.seed(42)

Using TensorFlow backend.


In [3]:
import tensorflow
tensorflow.__version__

'1.5.0'

In [4]:
SIZE = 512
path_to_train = 'data/train/'
data = pd.read_csv('data/train.csv')

In [5]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [6]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
        np.array(image_red_ch), 
        np.array(image_green_ch), 
        np.array(image_blue_ch)), -1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug
    

In [7]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1, 1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    return model

In [8]:
def fbeta_score_macro(y_true, y_pred, beta=1, threshold=0.2):

    y_true = K.cast(y_true, 'float')
    y_pred = K.cast(K.greater(K.cast(y_pred, 'float'), threshold), 'float')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)

    return K.mean(f1)

In [9]:
# create callbacks list
epochs = 100
batch_size = 32
checkpoint = ModelCheckpoint('data/InceptionV3.h5', monitor='val_loss', verbose=1, save_best_only=True, 
                             mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [10]:
# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)

In [11]:
# create train and valid datagens
train_generator = data_generator.create_train(train_dataset_info[train_indexes], batch_size, (SIZE, SIZE, 3), augument=True)
validation_generator = data_generator.create_train(train_dataset_info[valid_indexes], 32, (SIZE, SIZE, 3), augument=False)

In [12]:
# warm up model
model = create_model(input_shape=(SIZE, SIZE, 3), n_out=28)

for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True

model.compile(loss='binary_crossentropy', optimizer=Adam(1e-03), metrics=['acc', fbeta_score_macro])

In [13]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=5, 
    verbose=1)

Epoch 1/5
826/826 [==============================] - 471s 570ms/step - loss: 0.1795 - acc: 0.9407 - fbeta_score_macro: 0.0627 - val_loss: 0.2202 - val_acc: 0.9383 - val_fbeta_score_macro: 0.0382
Epoch 2/5
826/826 [==============================] - 434s 525ms/step - loss: 0.1684 - acc: 0.9431 - fbeta_score_macro: 0.0690 - val_loss: 0.3157 - val_acc: 0.9263 - val_fbeta_score_macro: 0.0340
Epoch 3/5
826/826 [==============================] - 430s 521ms/step - loss: 0.1657 - acc: 0.9437 - fbeta_score_macro: 0.0783 - val_loss: 0.3542 - val_acc: 0.9260 - val_fbeta_score_macro: 0.0316
Epoch 4/5
826/826 [==============================] - 430s 521ms/step - loss: 0.1651 - acc: 0.9439 - fbeta_score_macro: 0.0827 - val_loss: 0.2560 - val_acc: 0.9252 - val_fbeta_score_macro: 0.0419
Epoch 5/5
826/826 [==============================] - 417s 504ms/step - loss: 0.1645 - acc: 0.9441 - fbeta_score_macro: 0.0842 - val_loss: 0.3414 - val_acc: 0.9181 - val_fbeta_score_macro: 0.0350


In [14]:
# train all layers
for layer in model.layers:
    layer.trainable = True
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy', fbeta_score_macro])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
batch_normalization_95 (Batc (None, 512, 512, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 14, 14, 2048)      21802784  
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 14, 14, 32)        65568     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              6423552   
__________

In [15]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)
    

Epoch 1/100
826/826 [==============================] - 870s 1s/step - loss: 0.1667 - acc: 0.9436 - fbeta_score_macro: 0.0593 - val_loss: 0.1726 - val_acc: 0.9438 - val_fbeta_score_macro: 0.0624

Epoch 00001: val_loss improved from inf to 0.17263, saving model to data/InceptionV3.h5
Epoch 2/100
826/826 [==============================] - 869s 1s/step - loss: 0.1560 - acc: 0.9484 - fbeta_score_macro: 0.0926 - val_loss: 0.1531 - val_acc: 0.9508 - val_fbeta_score_macro: 0.1044

Epoch 00002: val_loss improved from 0.17263 to 0.15310, saving model to data/InceptionV3.h5
Epoch 3/100
826/826 [==============================] - 867s 1s/step - loss: 0.1477 - acc: 0.9503 - fbeta_score_macro: 0.1287 - val_loss: 0.1639 - val_acc: 0.9457 - val_fbeta_score_macro: 0.1146

Epoch 00003: val_loss did not improve from 0.15310
Epoch 4/100
826/826 [==============================] - 868s 1s/step - loss: 0.1424 - acc: 0.9509 - fbeta_score_macro: 0.1516 - val_loss: 0.1845 - val_acc: 0.9370 - val_fbeta_score_macr

Epoch 31/100
826/826 [==============================] - 869s 1s/step - loss: 0.0839 - acc: 0.9706 - fbeta_score_macro: 0.3506 - val_loss: 0.0840 - val_acc: 0.9702 - val_fbeta_score_macro: 0.3484

Epoch 00031: val_loss did not improve from 0.08381
Epoch 32/100
826/826 [==============================] - 867s 1s/step - loss: 0.0835 - acc: 0.9707 - fbeta_score_macro: 0.3505 - val_loss: 0.0838 - val_acc: 0.9701 - val_fbeta_score_macro: 0.3560

Epoch 00032: val_loss improved from 0.08381 to 0.08380, saving model to data/InceptionV3.h5
Epoch 33/100
826/826 [==============================] - 868s 1s/step - loss: 0.0829 - acc: 0.9709 - fbeta_score_macro: 0.3518 - val_loss: 0.0825 - val_acc: 0.9710 - val_fbeta_score_macro: 0.3540

Epoch 00033: val_loss improved from 0.08380 to 0.08252, saving model to data/InceptionV3.h5
Epoch 34/100
826/826 [==============================] - 868s 1s/step - loss: 0.0823 - acc: 0.9712 - fbeta_score_macro: 0.3517 - val_loss: 0.0829 - val_acc: 0.9708 - val_fbeta_sc

In [16]:
# Create submit
submit = pd.read_csv('data/sample_submission.csv')
predicted = []
draw_predict = []
model.load_weights('data/InceptionV3.h5')
for name in tqdm(submit['Id']):
    path = os.path.join('data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict >= 0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
np.save('data/draw_predict_InceptionV3_512.npy', score_predict)
submit.to_csv('data/submit_InceptionV3_512.csv', index=False)

100%|██████████| 11702/11702 [09:31<00:00, 20.47it/s]


In [18]:
df = pd.read_csv("data/submit_InceptionV3_512.csv")
df.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 23 25
